In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings

from pinecone import Pinecone
from openai import OpenAI
import numpy as np
import tiktoken
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/fjp/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
embeddings = OpenAIEmbeddings()
embed_model = "text-embedding-3-small"
openai_client = OpenAI()

/Users/fjp/.pyenv/versions/3.9.4/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
)

In [4]:
def get_embedding(text, model="text-embedding-3-small"):
    # Call the OpenAI API to get the embedding for the text
    response = openai_client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

def cosine_similarity_between_words(sentence1, sentence2):
    # Get embeddings for both words
    embedding1 = np.array(get_embedding(sentence1))
    embedding2 = np.array(get_embedding(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    print("Embedding for Sentence 1:", embedding1)
    print("\nEmbedding for Sentence 2:", embedding2)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like walking to the office"


similarity = cosine_similarity_between_words(sentence1, sentence2)
print(f"\n\nCosine similarity between '{sentence1}' and '{sentence2}': {similarity:.4f}")

Embedding for Sentence 1: [[-0.0008688  -0.05343446 -0.01759788 ...  0.03625558 -0.02790028
  -0.00312861]]

Embedding for Sentence 2: [[ 0.01854512 -0.01367737  0.01343909 ...  0.00889812 -0.04234606
  -0.00459543]]


Cosine similarity between 'I like walking to the park' and 'I like walking to the office': 0.7019


In [5]:
loader = WebBaseLoader("https://www.ratemyprofessors.com/professor/2894450")
data = loader.load()

print(data)

[Document(metadata={'source': 'https://www.ratemyprofessors.com/professor/2894450', 'title': 'S.K. Ramesh at California State University Northridge | Rate My Professors', 'description': 'S.K. Ramesh  is a professor in the Engineering department at California State University Northridge - see what their students are saying about them or leave a rating yourself.', 'language': 'No language found.'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\nS.K. Ramesh at California State University Northridge | Rate My Professors\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n ProfessorscancelatLog InSign Up5/ 5Jump To Ratings Overall Quality Based on\xa01\xa0ratingsS.K. Ramesh Professor in the Engineering department at California State University Northridge100%Would take again3Level of DifficultyRateCompareI'm Professor RameshProfessor Ramesh's Top TagsGet ready to readInspirationalRespected1 Student RatingsAll courses ECE578Dec 17th, 2023Quality5.0Difficulty3.0 ECE578Dec 17th, 2023For Credit: YesWould Take Again: YesG

In [6]:
texts = text_splitter.split_documents(data)

In [7]:
texts

[Document(metadata={'source': 'https://www.ratemyprofessors.com/professor/2894450', 'title': 'S.K. Ramesh at California State University Northridge | Rate My Professors', 'description': 'S.K. Ramesh  is a professor in the Engineering department at California State University Northridge - see what their students are saying about them or leave a rating yourself.', 'language': 'No language found.'}, page_content="S.K. Ramesh at California State University Northridge | Rate My Professors\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n ProfessorscancelatLog InSign Up5/ 5Jump To Ratings Overall Quality Based on\xa01\xa0ratingsS.K. Ramesh Professor in the Engineering department at California State University Northridge100%Would take again3Level of DifficultyRateCompareI'm Professor RameshProfessor Ramesh's Top TagsGet ready to readInspirationalRespected1 Student RatingsAll courses ECE578Dec 17th, 2023Quality5.0Difficulty3.0 ECE578Dec 17th, 2023For Credit: YesWould Take Again: YesGrade: ATextbook: YesOnline C

In [8]:
vectorstore = PineconeVectorStore(index_name="rag", embedding=embeddings)

index_name = "rag"

namespace = "ns1"

In [ ]:
for document in texts:
    print("\n\n\n\n----")
    print(document.metadata , document.page_content)
    print("\n\n\n\n----")